In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
#import matplotlib.pyplot as plt
import time
import os
import copy
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler
from train_snippets import train_model


In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
train_set = ImageFolder('../../datasets/type_dataset/train_vehicle_types/', transform=data_transforms['train'])
print(train_set)

Dataset ImageFolder
    Number of datapoints: 36960
    Root Location: ../../datasets/type_dataset/train_vehicle_types/
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None


In [4]:
train_size = int(0.8 * len(train_set))
val_size = len(train_set) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_set, [train_size, val_size])
train_dataset.dataset = copy.copy(train_set) #copy, so it can have other transformation.
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

dataloaders = {}
dataloaders['train'] = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
dataloaders['val'] = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8)

In [5]:
dataloaders['train'].dataset.dataset

Dataset ImageFolder
    Number of datapoints: 36960
    Root Location: ../../datasets/type_dataset/train_vehicle_types/
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None

In [6]:
model_ft = models.resnet50(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters())

# Decay LR by a factor of 0.1 every 10 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [9]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,dataloaders=dataloaders,
                       num_epochs=50, device=device)

Epoch 0/49
----------


RuntimeError: The size of tensor a (5) must match the size of tensor b (64) at non-singleton dimension 1

In [ ]:
model_ft

# Test

In [16]:
model = model_ft
model.eval()   # Set model to evaluate mode
running_loss = 0.0
running_corrects = 0
dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}

for inputs, labels in dataloaders['val']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

    # statistics
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

epoch_loss = running_loss / dataset_sizes['val']
epoch_acc = running_corrects.double() / dataset_sizes['val']

In [17]:
print('val loss:', epoch_loss,', val acc:', epoch_acc)

val loss: 0.51838153549087 , val acc: tensor(0.7896, device='cuda:0', dtype=torch.float64)


In [18]:
x = torch.load('./best_train_val_0.8/model_best.pth.tar')

In [20]:
x.keys()

dict_keys(['epoch', 'state_dict', 'best_acc', 'best_loss', 'optimizer'])

In [1]:
torch.mea

NameError: name 'torch' is not defined